In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import accuracy_score,classification_report,roc_curve,roc_auc_score,confusion_matrix,precision_recall_curve,precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold,cross_val_predict
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder,FunctionTransformer
from imblearn.under_sampling import RandomUnderSampler,NearMiss,TomekLinks,InstanceHardnessThreshold
from sklearn.compose import ColumnTransformer
import optuna
from sklearn.metrics import average_precision_score
from sklearn.utils import shuffle
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE,ADASYN
from sklearn.metrics import average_precision_score

from sklearn.preprocessing import FunctionTransformer
import joblib
from custom_transformers import log_abs_transform
from sklearn.preprocessing import PowerTransformer

In [2]:
df = pd.read_csv('fraud.csv')

In [3]:
df.shape

(6362620, 11)

In [4]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [5]:
df = df[~(df['nameDest'].str.startswith('M'))|(df['nameOrig'].str.startswith('M'))]

In [6]:
df.shape

(4211125, 11)

In [7]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.00,1,0
9,1,DEBIT,5337.77,C712410124,41720.0,36382.23,C195600860,41898.0,40348.79,0,0
10,1,DEBIT,9644.94,C1900366749,4465.0,0.00,C997608398,10845.0,157982.12,0,0
15,1,CASH_OUT,229133.94,C905080434,15325.0,0.00,C476402209,5083.0,51513.44,0,0


In [8]:
df.drop(columns=['nameOrig','nameDest'],inplace=True)

In [9]:
df.head(2)

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.0,181.0,0.0,0.0,0.0,1,0
3,1,CASH_OUT,181.0,181.0,0.0,21182.0,0.0,1,0


In [10]:
df['diffOrg'] = df['newbalanceOrig'] - df['oldbalanceOrg']
df['diffdest'] = df['newbalanceDest'] - df['oldbalanceDest']
df['isFraud'] = df['isFraud'].astype('int')
df.drop(columns=['oldbalanceOrg','newbalanceOrig','newbalanceDest','oldbalanceDest'],inplace=True)
hours = df['step'] % 24
bins = [0, 6, 12, 18, 24]
labels = ['Late Night', 'Morning', 'Afternoon', 'Night']

buckets =  pd.cut(
    hours,
    bins=bins,
    labels=labels,
    right=False  # 0-5, 6-11, 12-17, 18-23
)
df['transaction_time'] = buckets

In [11]:
df.drop(columns=['step'],inplace=True)

In [12]:
df.drop(columns=['isFlaggedFraud'],inplace=True)

In [13]:
df.head()

,type,amount,isFraud,diffOrg,diffdest,transaction_time
2,TRANSFER,181.00,1,-181.00,0.00,Late Night
3,CASH_OUT,181.00,1,-181.00,-21182.00,Late Night
9,DEBIT,5337.77,0,-5337.77,-1549.21,Late Night
10,DEBIT,9644.94,0,-4465.00,147137.12,Late Night
15,CASH_OUT,229133.94,0,-15325.00,46430.44,Late Night


In [14]:
# Create train , test , validation Dataset
train,val = train_test_split(df,stratify=df['isFraud'],test_size=0.1,random_state=42)

In [15]:
train['isFraud'].value_counts()/train.shape[0]

isFraud
0    0.99805
1    0.00195
Name: count, dtype: float64

In [16]:
val['isFraud'].value_counts()/val.shape[0]

isFraud
0    0.99805
1    0.00195
Name: count, dtype: float64

In [19]:
# Save Validation data
# val.to_csv('validation_set.csv',index=False)

In [20]:
# save training and testing data
# train.to_csv('train_set.csv',index=False)

In [17]:
# using 25% random non_fraud data as for sample
fraud_df = train[train['isFraud']==1]
non_fraud_df = train[train['isFraud']==False].sample(int(len(train) * 0.25),random_state=42)
sample_df = pd.concat([fraud_df, non_fraud_df])
sampel_df = sample_df.sample(len(sample_df))
sample_df.shape

(954895, 6)

In [18]:
x,y = sample_df.drop(columns=['isFraud']),sample_df['isFraud']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [19]:
# log_transformer = FunctionTransformer(
#     log_abs_transform,
#     feature_names_out="one-to-one"
# )
pt = PowerTransformer(method="yeo-johnson")

In [20]:
# preprocessor
preprocessor = ColumnTransformer(transformers=[
                                              ('num',pt,['amount','diffOrg','diffdest']),
                                              ('cat',OneHotEncoder(),['type'])
                                              ])


In [21]:
# create model
sampler = SMOTE(sampling_strategy=0.2, random_state=42)
lgbm = LGBMClassifier(random_state=42)
pipeline = Pipeline([
      ('preprocessing',preprocessor),
      ('sampling',sampler),
      ('model',lgbm)
])

In [ ]:
# model training and threshold selection
pipeline.fit(x_train,y_train)
y_train_proba = pipeline.predict_proba(x_train)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_train, y_train_proba)
f1_scores = 2 * precision * recall / (precision + recall + 1e-12)
best_idx = np.argmax(f1_scores[:-1])
best_threshold = thresholds[best_idx]
print(f"Chosen Threshold: {best_threshold:.4f}")

In [197]:
# model performance on test data with best_threshold
y_test_proba = pipeline.predict_proba(x_test)[:, 1]
y_test_pred = (y_test_proba >= best_threshold).astype(int)

print("Accuracy:", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))


C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Accuracy: 0.9981431236592261
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    227485
           1       0.97      0.72      0.82      1394

    accuracy                           1.00    228879
   macro avg       0.98      0.86      0.91    228879
weighted avg       1.00      1.00      1.00    228879



In [198]:
print(confusion_matrix(y_test,y_test_pred))

[[227453     32]
 [   393   1001]]


# Tunning

In [144]:
def objective(trial):
  # Define Parameters
  params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 120),
        'max_depth': trial.suggest_int('max_depth', -1, 15),
        'min_child_samples':  trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
        'n_estimators': trial.suggest_int('n_estimators',100,1000,step=100),
        'n_jobs': -1,
        'random_state': 42
        
    }

    

    

  # log_transformer = FunctionTransformer(
  #   lambda x: np.log1p(np.abs(x)),
  #   feature_names_out="one-to-one"
  #   )

  pt = PowerTransformer(method="yeo-johnson")    
    

  # Column transformer
  preprocessor = ColumnTransformer(transformers=[
                                              ('num',pt,['amount','diffOrg','diffdest']),
                                              ('cat',OneHotEncoder(),['type','transaction_time'])
                                               ])


  # create model
  lgbm = LGBMClassifier(**params)
  pipeline = Pipeline([
      ('preprocessing',preprocessor),
      ('sampling',SMOTE(sampling_strategy=0.2, random_state=42)),
      ('model',lgbm)
  ])

  cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

  y_pred_proba = cross_val_predict(pipeline,x_train,y_train,cv=cv,method='predict_proba')[:,1]
   
  # Find best threshold INSIDE CV
  precision, recall, thresholds = precision_recall_curve(y_train, y_pred_proba)
  f1_scores = 2 * precision * recall / (precision + recall + 1e-12)

  best_f1 = np.max(f1_scores[:-1])  # ignore last threshold
  return best_f1


In [145]:
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=30,show_progress_bar=False)

[I 2026-01-14 14:09:59,577] A new study created in memory with name: no-name-010aeb51-cb06-4bae-addc-75c54b8dc165


[LightGBM] [Info] Number of positive: 121269, number of negative: 606345
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.059021 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 939
[LightGBM] [Info] Number of data points in the train set: 727614, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166667 -> initscore=-1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 121269, number of negative: 606345
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020634 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 947
[LightGBM] [Info] Number of data points in the train set: 727614, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166667 -> initscore=-1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 121269, number of negative: 606345
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027142 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 947
[LightGBM] [Info] Number of data points in the train set: 727614, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166667 -> initscore=-1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 121269, number of negative: 606346
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021918 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 927
[LightGBM] [Info] Number of data points in the train set: 727615, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166666 -> initscore=-1.609440
[LightGBM] [Info] Start training from score -1.609440
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2026-01-14 16:18:57,967] Trial 29 finished with value: 0.8521262002738531 and parameters: {'learning_rate': 0.15239219841734283, 'num_leaves': 46, 'max_depth': 9, 'min_child_samples': 47, 'subsample': 0.8123294753645353, 'colsample_bytree': 0.6304621296979604, 'reg_alpha': 4.638586146103359, 'reg_lambda': 4.4093135634928355, 'n_estimators': 600}. Best is trial 18 with value: 0.8533651638963006.


In [146]:
print(study.best_params)
print(study.best_value)

{'learning_rate': 0.16906548087764994, 'num_leaves': 56, 'max_depth': 6, 'min_child_samples': 50, 'subsample': 0.8540198726215833, 'colsample_bytree': 0.8357010407356671, 'reg_alpha': 4.948923359868166, 'reg_lambda': 3.766283078813302, 'n_estimators': 700}
0.8533651638963006


# Model With Best Parameteres

In [218]:
# log_transformer = FunctionTransformer(
#     log_abs_transform,
#     feature_names_out="one-to-one"
# )
pt = PowerTransformer(method="yeo-johnson")

In [219]:
# preprocessor
preprocessor = ColumnTransformer(transformers=[
                                              ('num',pt,['amount','diffOrg','diffdest']),
                                              ('cat',OneHotEncoder(),['type','transaction_time'])
                                              ])


In [220]:
# create model
# best_params = study.best_params
# best_params = {'learning_rate': 0.16906548087764994, 'num_leaves': 56,
#                'max_depth': 6, 'min_child_samples': 50, 'subsample': 0.8540198726215833, 
#                'colsample_bytree': 0.8357010407356671, 'reg_alpha': 4.948923359868166, 'reg_lambda': 3.766283078813302, 'n_estimators': 700}
best_params = {'learning_rate':0.0545, 'num_leaves': 74,
               'max_depth': 10, 'min_child_samples': 91, 'subsample':0.401, 
               'colsample_bytree': 0.787, 'reg_alpha': 1.209, 'reg_lambda': 4.968, 'n_estimators': 300}


sampler = SMOTE(sampling_strategy=0.2, random_state=42)
lgbm = LGBMClassifier(**best_params,random_state=42)
pipeline = Pipeline([
      ('preprocessing',preprocessor),
      ('sampling',sampler),
      ('model',lgbm)
])

pipeline.fit(x_train,y_train)

[LightGBM] [Info] Number of positive: 181903, number of negative: 909518
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008818 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 829
[LightGBM] [Info] Number of data points in the train set: 1091421, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.166666 -> initscore=-1.609441
[LightGBM] [Info] Start training from score -1.609441
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', PowerTransformer(),
                                                  ['amount', 'diffOrg',
                                                   'diffdest']),
                                                 ('cat', OneHotEncoder(),
                                                  ['type',
                                                   'transaction_time'])])),
                ('sampling', SMOTE(random_state=42, sampling_strategy=0.2)),
                ('model',
                 LGBMClassifier(colsample_bytree=0.787, learning_rate=0.0545,
                                max_depth=10, min_child_samples=91,
                                n_estimators=300, num_leaves=74,
                                random_state=42, reg_alpha=1.209,
                                reg_lambda=4.968, subsample=0.401))])

In [221]:
# model training with best parameters  and threshold selection

y_train_proba = pipeline.predict_proba(x_train)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_train, y_train_proba)
f1_scores = 2 * precision * recall / (precision + recall + 1e-12)
best_idx = np.argmax(f1_scores[:-1])
best_threshold = thresholds[best_idx]

print(f"Chosen Threshold: {best_threshold:.4f}")

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Chosen Threshold: 0.9497


In [222]:
# model performance on test data with best_threshold
y_test_proba = pipeline.predict_proba(x_test)[:, 1]
y_test_pred = (y_test_proba >= best_threshold).astype(int)

print("Accuracy:", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Accuracy: 0.9983790561825244
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    227485
           1       0.95      0.77      0.85      1394

    accuracy                           1.00    228879
   macro avg       0.98      0.89      0.93    228879
weighted avg       1.00      1.00      1.00    228879



In [224]:
print(confusion_matrix(y_test,y_test_pred))

[[227431     54]
 [   317   1077]]


# Model Performance on validation data

In [225]:
val = pd.read_csv('validation_set.csv')

In [226]:
val.head()

,type,amount,isFraud,diffOrg,diffdest,transaction_time
0,CASH_IN,406312.84,0,406312.84,-406312.84,Afternoon
1,CASH_IN,308531.31,0,308531.31,-308531.31,Afternoon
2,CASH_OUT,41998.10,0,0.00,41998.10,Night
3,CASH_IN,25811.48,0,25811.48,-25811.48,Afternoon
4,CASH_OUT,312009.00,0,-31972.00,312009.00,Morning


In [227]:
val.shape

(421113, 6)

In [228]:
x_val,y_val = val.drop(columns=['isFraud']),val['isFraud']
x_val.shape,y_val.shape

((421113, 5), (421113,))

In [230]:
y_test_proba = pipeline.predict_proba(x_val)[:, 1]
y_test_pred = (y_test_proba >= best_threshold).astype(int)

print("Accuracy:", accuracy_score(y_val, y_test_pred))
print(classification_report(y_val, y_test_pred))

C:\Users\Yogesh\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Accuracy: 0.9993042247567755
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    420292
           1       0.84      0.79      0.82       821

    accuracy                           1.00    421113
   macro avg       0.92      0.89      0.91    421113
weighted avg       1.00      1.00      1.00    421113



In [231]:
print(confusion_matrix(y_val,y_test_pred))

[[420173    119]
 [   174    647]]


Insight:

## The model performs similarly on training and validation data, indicating no overfitting. The model achieves 84% precision, which means when it predicts a transaction as fraud,it is correct 84 times out of 100. This is good because any transaction flagged as fraud can be investigated with high confidence.
## The recall is 79%, meaning the model is able to detect 78 out of 100 actual fraud cases. This shows that the fraud-catching ability is moderate and can be improved further.

## In short, although the model misses some fraud cases, when it does flag a transaction, there is a high probability that it is truly fraudulent.

# Export Complete Pipeline

In [119]:
joblib.dump(pipeline, "fraud_pipeline.joblib")

['fraud_pipeline.joblib']